In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import backend as K

# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize and flatten images
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Define VAE architecture
input_dim = x_train.shape[1]
latent_dim = 2  # 2-dimensional latent space

# Encoder
input_img = Input(shape=(input_dim,))
encoded = Dense(256, activation='relu')(input_img)
z_mean = Dense(latent_dim)(encoded)
z_log_var = Dense(latent_dim)(encoded)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = Input(shape=(latent_dim,))
decoded = Dense(256, activation='relu')(decoder_input)
output_img = Dense(input_dim, activation='sigmoid')(decoded)

# Define VAE model
encoder = Model(input_img, z_mean)
decoder = Model(decoder_input, output_img)

# VAE model
vae_output = decoder(z)
vae = Model(input_img, vae_output)

# Compute VAE loss
reconstruction_loss = keras.losses.binary_crossentropy(input_img, vae_output)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

# Train the VAE
epochs = 100
batch_size = 128
history = vae.fit(x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, None))

# Generate latent space points using VAE
latent_points_vae = encoder.predict(x_test)

# Generate samples from Dirichlet distribution
samples = np.random.dirichlet(np.ones(latent_dim), size=10)

# Visualize the latent space after 100 epochs
plt.figure(figsize=(6, 6))
plt.subplot(1, 2, 1)
plt.scatter(latent_points_vae[:, 0], latent_points_vae[:, 1], c='b', cmap='rainbow')
plt.title('Latent Space Visualization (VAE)')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')

# Generate and visualize samples from the VAE after 100 epochs
decoded_samples = decoder.predict(samples)
decoded_samples = decoded_samples.reshape(-1, 28, 28)

plt.subplot(1, 2, 2)
for i in range(10):
    plt.imshow(decoded_samples[i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title('Generated Samples (VAE)')
    plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K

# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize and reshape images
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Define VAE architecture
input_shape = x_train.shape[1:]
latent_dim = 2  # 2-dimensional latent space

# Encoder
inputs = Input(shape=input_shape)
x = Conv2D(32, 3, activation='relu', strides=2, padding='same')(inputs)
x = Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = Flatten()(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(7 * 7 * 64, activation='relu')(decoder_inputs)
x = Reshape((7, 7, 64))(x)
x = Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
outputs = Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)

# Define VAE model
encoder = Model(inputs, z_mean)
decoder = Model(decoder_inputs, outputs)
vae_inputs = inputs
vae_outputs = decoder(encoder(vae_inputs))
vae = Model(vae_inputs, vae_outputs)

# Compute VAE loss
reconstruction_loss = keras.losses.binary_crossentropy(K.flatten(vae_inputs), K.flatten(vae_outputs))
reconstruction_loss *= input_shape[0] * input_shape[1]
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

# Train the VAE
epochs = 100
batch_size = 128
history = vae.fit(x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, None))

# Generate latent space points using VAE
latent_points_vae = encoder.predict(x_test)

# Generate samples from Dirichlet distribution
samples = np.random.dirichlet(np.ones(latent_dim), size=10)

# Visualize the latent space after 100 epochs
plt.figure(figsize=(6, 6))
plt.subplot(1, 2, 1)
plt.scatter(latent_points_vae[:, 0], latent_points_vae[:, 1], c='b', cmap='rainbow')
plt.title('Latent Space Visualization (VAE)')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')

# Generate and visualize samples from the VAE after 100 epochs
decoded_samples = decoder.predict(samples)
decoded_samples = decoded_samples.reshape(-1, 28, 28)

plt.subplot(1, 2, 2)
for i in range(10):
    plt.imshow(decoded_samples[i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title('Generated Samples (VAE)')
plt.tight_layout()
plt.show()


In [ ]:
# Define VAE architecture
input_dim = x_train.shape[1]
latent_dim = 2  # 2-dimensional latent space

# Encoder
input_img = Input(shape=(input_dim,))
encoded = Dense(256, activation='relu')(input_img)
z_mean = Dense(latent_dim)(encoded)
z_log_var = Dense(latent_dim)(encoded)

# Reparameterization trick
defsampling(args):
    z_mean,z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim), mean=0.,stddev=1.0)
    returnz_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean,z_log_var])

# Decoder
decoder_input = Input(shape=(latent_dim,))
decoded = Dense(256, activation='relu')(decoder_input)
output_img = Dense(input_dim, activation='sigmoid')(decoded)


In [ ]:
# Define VAE architecture
input_shape = x_train.shape[1:]
latent_dim = 2  # 2-dimensional latent space

# Encoder
inputs = Input(shape=input_shape)
x = Conv2D(32,3, activation='relu', strides=2, padding='same')(inputs)
x = Conv2D(64,3, activation='relu', strides=2, padding='same')(x)
x = Flatten()(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Reparameterization trick
defsampling(args):
    z_mean,z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim), mean=0.,stddev=1.0)
    returnz_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean,z_log_var])

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(7 * 7 * 64, activation='relu')(decoder_inputs)
x = Reshape((7,7,64))(x)
x = Conv2DTranspose(64,3, activation='relu', strides=2, padding='same')(x)
x = Conv2DTranspose(32,3, activation='relu', strides=2, padding='same')(x)
outputs = Conv2DTranspose(1,3, activation='sigmoid', padding='same')(x)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load Fashion MNIST dataset
(train_images, _), (test_images, _) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize and reshape data
train_images = train_images.astype('float32') / 255.0
train_images = np.reshape(train_images, (-1, 28, 28, 1))

# Define Conv2D VAE architecture
latent_dim = 2

class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim), mean=0.0, stddev=1.0)
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

encoder_inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = tf.keras.layers.Flatten()(x)
z_mean = tf.keras.layers.Dense(latent_dim)(x)
z_log_var = tf.keras.layers.Dense(latent_dim)(x)
z = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')

decoder_inputs = tf.keras.Input(shape=(latent_dim,))
x = tf.keras.layers.Dense(7 * 7 * 64, activation='relu')(decoder_inputs)
x = tf.keras.layers.Reshape((7, 7, 64))(x)
x = tf.keras.layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
decoder_outputs = tf.keras.layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)
decoder = tf.keras.Model(decoder_inputs, decoder_outputs, name='decoder')

# Create VAE model
vae_inputs = encoder_inputs
vae_outputs = decoder(encoder(encoder_inputs)[2])
vae_model = tf.keras.Model(vae_inputs, vae_outputs, name='vae')


def loss_func(encoder_mu, encoder_log_variance):
    def vae_reconstruction_loss(y_true, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = tf.keras.backend.mean(tf.keras.backend.square(y_true-y_predict), axis=[1, 2, 3])
        return reconstruction_loss_factor * reconstruction_loss

    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * tf.keras.backend.sum(1.0 + encoder_log_variance - tf.keras.backend.square(encoder_mu) - tf.keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * tf.keras.backend.sum(1.0 + encoder_log_variance - tf.keras.backend.square(encoder_mu) - tf.keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_loss(y_true, y_predict):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss

# Compile the model
vae_model.compile(optimizer='adam', loss=loss_func(z_mean, z_log_var))
#vae.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.0005), loss=loss_func(encoder_mu, encoder_log_variance))
# Training the model
 #vae_model.fit(train_images, train_images, epochs=30, batch_size=128)
history = vae_model.fit(train_images, train_images, epochs=20, batch_size=32, shuffle=True)
# Plot the loss curve
plt.plot(history.history['loss'])
plt.title('VAE Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Generate samples of "Trouser" and "Shirt" classes
num_samples = 10

# Generate "Trouser" samples
z_samples_trouser = np.random.normal(size=(num_samples, latent_dim))
generated_images_trouser = decoder.predict(z_samples_trouser)

# Generate "Shirt" samples
z_samples_shirt = np.random.normal(size=(num_samples, latent_dim))
generated_images_shirt = decoder.predict(z_samples_shirt)

# Plot generated samples
plt.figure(figsize=(15, 4))
for i in range(num_samples):
    plt.subplot(2, num_samples, i + 1)
    plt.imshow(generated_images_trouser[i].reshape(28, 28), cmap='gray')
    plt.title('Trouser')
    plt.axis('off')

    plt.subplot(2, num_samples, num_samples + i + 1)
    plt.imshow(generated_images_shirt[i].reshape(28, 28), cmap='gray')
    plt.title('Shirt')
    plt.axis('off')

plt.show()
